# USCIS Credible Fear Spreadsheets

## Purpose 

This notebook provides functionality to extract data data from the [USCIS](https://www.uscis.gov/tools/reports-and-studies/semi-monthly-credible-fear-and-reasonable-fear-receipts-and-decisions) spreadsheets on credible and reasonable fear interviews. This data is released by USCIS twice monthly and features different data tables embedded in a single excel spreadsheet worksheet. This data can be time consuming to work with given that the data is reported in two week increments vs monthly totals and it has rolling time period, meaning that each new file has fewer historical months of data. 

<img src="../misc/images/uscis_webpage.png" width=900 />


The USCIS only displays the most recent data on the their webpage, but other files are available by modifying the download url. 

```https://www.uscis.gov/sites/default/files/document/reports/Congressional-Semi-Monthly-Report-11-01-20-to-11-15-21.xlsx```


## Approach

We will use python to download all available files, then  extract example subsets of data across the history of these spreadsheets and summarize by month. 

## The Code

**Imports**

In [111]:
import requests
import pandas as pd
import numpy as np
from pathlib import Path

pd.set_option("max_columns", 100)
pd.set_option("display.max_colwidth", 0)

## 1. Downloading Files

Visiting the source link where the current version of this file is located we can right click on the file link and `copy link address`. 
You should get something like the following (note USCIS may have updated data in the meantime so exact month may differ): 

https://www.uscis.gov/sites/default/files/document/reports/Congressional-Semi-Monthly-Report-11-01-20-to-11-15-21.xlsx
  
We can split the url into a base part:  `https://www.uscis.gov/sites/default/files/document/reports/` <br>
and a filename part:`Congressional-Semi-Monthly-Report-11-01-20-to-11-15-21.xlsx`

In [177]:
base_url = "https://www.uscis.gov/sites/default/files/document/reports"
rawfile_output_directory = Path(
    "../data/raw_source_files/uscis-cfr-semi-monthly-reports/"
)

We will use code to download all available `.xlsx` files

In [113]:
def download_file(base_url, filename, output_directory):

    """
    Function to download a single file from a webpage. Assumes that the file
    is located on a webpage where they download path is a base_url/filename
    type structure. Code could be adapted for other stuctures.

    Parameters:
      url_base: base Url of page where files are linked/located
      filename: filename of the file / second part of the url
      output_folder: Folder path to savae file

    Returns:
      Saves the file to the output directory, function itself returns nothing.

    Example:
        download_file('https://www.uscis.gov/sites/default/files/document/reports',
                      'Congressional-Semi-Monthly-Report-06-01-20-to-06-15-21.xlsx'
                      '.'
        )

    """

    full_url = f"{base_url}/{filename}"

    response = requests.get(str(full_url))
    if response.status_code == 200:
        # Write content to file
        output_directory = Path(output_directory)
        output_directory.mkdir(exist_ok=True, parents=True)
        outpath = output_directory / f"{filename}"
        pdf = open(str(outpath), "wb")
        pdf.write(response.content)
        pdf.close()
        print(full_url)
    else:
        print("File not found:", full_url)

We will iterate over the years (2020-->20, and 2021 --> 21) and all numeric month labels and try to download the data. If a file is not available the `download_file` function will print out that the file was not found. This is to be expected in some cases as the USCIS has not released files for every month in 2020 and 2021.

In [114]:
# Just try 2020 and 2021
for year_i in range(20, 21):
    # let's just try all months from 1-> 12
    for month_i in range(1, 12 + 1):
        month = str(month_i).zfill(2)
        download_file(
            base_url,
            f"Congressional-Semi-Monthly-Report-{month}-01-{year_i}-to-{month}-15-{year_i+1}.xlsx",
            rawfile_output_directory,
        )

https://www.uscis.gov/sites/default/files/document/reports/Congressional-Semi-Monthly-Report-01-01-20-to-01-15-21.xlsx
https://www.uscis.gov/sites/default/files/document/reports/Congressional-Semi-Monthly-Report-02-01-20-to-02-15-21.xlsx
File not found: https://www.uscis.gov/sites/default/files/document/reports/Congressional-Semi-Monthly-Report-03-01-20-to-03-15-21.xlsx
https://www.uscis.gov/sites/default/files/document/reports/Congressional-Semi-Monthly-Report-04-01-20-to-04-15-21.xlsx
https://www.uscis.gov/sites/default/files/document/reports/Congressional-Semi-Monthly-Report-05-01-20-to-05-15-21.xlsx
https://www.uscis.gov/sites/default/files/document/reports/Congressional-Semi-Monthly-Report-06-01-20-to-06-15-21.xlsx
File not found: https://www.uscis.gov/sites/default/files/document/reports/Congressional-Semi-Monthly-Report-07-01-20-to-07-15-21.xlsx
https://www.uscis.gov/sites/default/files/document/reports/Congressional-Semi-Monthly-Report-08-01-20-to-08-15-21.xlsx
https://www.usci

We saved all files to the `output_directory_path`, why don't you go look in that folder and verify that those files are there and take a look at a few quickly. 

## 2. Parse an example file 

Since the excel files contain multiple tables in a single sheet we need to use a function to try to parse those different tables into independent dataframes. Thankfully we have found such a function and apply it below. Note we are not explaining this function in too much detail as it is somewhat advacnced but understand what it generates should be relatively straightforward. 

In [187]:
# Parsing function
def parse_excel_sheet(file, sheet_name=0):
    """
    Parses multiple tables from an excel sheet into multiple data frame objects. Returns [dfs, df_mds],
    where dfs is a list of data frames and df_mds their potential associated metadata

    Parameters:
        file: path and filename to excel sheet
        sheet_name: Which worksheet to will default to first worksheet

    Returns:
        list of dataframes, list of metadata (columns) of the dataframes

    source: https://stackoverflow.com/questions/43367805/pandas-read-excel-multiple-tables-on-the-same-sheet
    """

    threshold = 3
    xl = pd.ExcelFile(file)
    entire_sheet = xl.parse(sheet_name=sheet_name)

    # count the number of non-Nan cells in each row and then the change in that number between adjacent rows
    n_values = np.logical_not(entire_sheet.isnull()).sum(axis=1)
    n_values_deltas = n_values[1:] - n_values[:-1].values

    # define the beginnings and ends of tables using delta in n_values
    table_beginnings = n_values_deltas > threshold
    table_beginnings = table_beginnings[table_beginnings].index
    table_endings = n_values_deltas < -threshold
    table_endings = table_endings[table_endings].index
    if (
        len(table_beginnings) < len(table_endings)
        or len(table_beginnings) > len(table_endings) + 1
    ):
        raise BaseException("Could not detect equal number of beginnings and ends")

    # look for metadata before the beginnings of tables
    md_beginnings = []
    for start in table_beginnings:
        md_start = n_values.iloc[:start][n_values == 0].index[-1] + 1
        md_beginnings.append(md_start)

    # make data frames
    dfs = []
    df_mds = []
    for ind in range(len(table_beginnings)):
        start = table_beginnings[ind] + 1
        if ind < len(table_endings):
            stop = table_endings[ind]
        else:
            stop = entire_sheet.shape[0]
        df = xl.parse(sheet_name=sheet_name, skiprows=start, nrows=stop - start)
        dfs.append(df)

        md = xl.parse(
            sheet_name=sheet_name,
            skiprows=md_beginnings[ind],
            nrows=start - md_beginnings[ind] - 1,
        ).dropna(axis=1)
        df_mds.append(md)
    return dfs, df_mds

In [116]:
test_file_path = (
    rawfile_output_directory
    / "Congressional-Semi-Monthly-Report-01-01-20-to-01-15-21.xlsx"
)

Let's see an example use of this function. We can call it on one file. The `dfs` variable will store a list of dataframes for the different tables in the first sheet of the `January xlsx` file. For reference we have included a screen shot below. 

<img src="../misc/images/uscis_multi_tables.png" width=900 />


In [188]:
dfs, _ = parse_excel_sheet(test_file_path)

**We get the first table - compare to image above**

In [189]:
dfs[0]

,Unnamed: 0,From,Unnamed: 2,2020-01-01 00:00:00,2020-01-16 00:00:00,2020-02-01 00:00:00,2020-02-16 00:00:00,2020-03-01 00:00:00,2020-03-16 00:00:00,2020-04-01 00:00:00,2020-04-16 00:00:00,2020-05-01 00:00:00,2020-05-16 00:00:00,2020-06-01 00:00:00,2020-06-16 00:00:00,2020-07-01 00:00:00,2020-07-16 00:00:00,2020-08-01 00:00:00,2020-08-16 00:00:00,2020-09-01 00:00:00,2020-09-16 00:00:00,2020-10-01 00:00:00,2020-10-16 00:00:00,2020-11-01 00:00:00,2020-11-16 00:00:00,2020-12-01 00:00:00,2020-12-16 00:00:00,2021-01-01 00:00:00
0,NaN,To,NaN,2020-01-15 00:00:00,2020-01-31 00:00:00,2020-02-15 00:00:00,2020-02-29 00:00:00,2020-03-15 00:00:00,2020-03-31 00:00:00,2020-04-15 00:00:00,2020-04-30 00:00:00,2020-05-15 00:00:00,2020-05-31 00:00:00,2020-06-15 00:00:00,2020-06-30 00:00:00,2020-07-15 00:00:00,2020-07-31 00:00:00,2020-08-15 00:00:00,2020-08-31 00:00:00,2020-09-15 00:00:00,2020-09-30 00:00:00,2020-10-15 00:00:00,2020-10-31 00:00:00,2020-11-15 00:00:00,2020-11-30 00:00:00,2020-12-15 00:00:00,2020-12-31 00:00:00,2021-01-15 00:00:00
1,NaN,Case Receipts,NaN,1877,2901,2097,2532,2055,1578,433,253,256,205,283,367,399,325,279,336,308,417,341,368,284,381,459,562,435
2,NaN,All Decisions,NaN,1316,2373,2419,2121,1866,2644,1002,704,467,214,251,313,240,470,412,276,239,259,499,446,242,335,410,388,571
3,NaN,Fear Established (Y),NaN,408,800,870,622,523,916,463,296,179,97,133,138,122,258,253,163,114,111,278,242,137,210,249,243,351
4,NaN,Fear Not Established (N),NaN,655,1236,1198,1199,992,1359,464,350,221,89,90,148,83,137,128,71,84,94,182,125,66,91,92,83,145
5,NaN,Administratively Closed,NaN,253,337,351,300,351,369,75,58,67,28,28,27,35,75,31,42,41,54,39,79,39,34,69,62,75


**We get the second table - compare to image above**

In [121]:
dfs[1]

,Unnamed: 0,From,Unnamed: 2,2020-01-01 00:00:00,2020-01-16 00:00:00,2020-02-01 00:00:00,2020-02-16 00:00:00,2020-03-01 00:00:00,2020-03-16 00:00:00,2020-04-01 00:00:00,2020-04-16 00:00:00,2020-05-01 00:00:00,2020-05-16 00:00:00,2020-06-01 00:00:00,2020-06-16 00:00:00,2020-07-01 00:00:00,2020-07-16 00:00:00,2020-08-01 00:00:00,2020-08-16 00:00:00,2020-09-01 00:00:00,2020-09-16 00:00:00,2020-10-01 00:00:00,2020-10-16 00:00:00,2020-11-01 00:00:00,2020-11-16 00:00:00,2020-12-01 00:00:00,2020-12-16 00:00:00,2021-01-01 00:00:00
0,NaN,To,NaN,2020-01-15 00:00:00,2020-01-31 00:00:00,2020-02-15 00:00:00,2020-02-29 00:00:00,2020-03-15 00:00:00,2020-03-31 00:00:00,2020-04-15 00:00:00,2020-04-30 00:00:00,2020-05-15 00:00:00,2020-05-31 00:00:00,2020-06-15 00:00:00,2020-06-30 00:00:00,2020-07-15 00:00:00,2020-07-31 00:00:00,2020-08-15 00:00:00,2020-08-31 00:00:00,2020-09-15 00:00:00,2020-09-30 00:00:00,2020-10-15 00:00:00,2020-10-31 00:00:00,2020-11-15 00:00:00,2020-11-30 00:00:00,2020-12-15 00:00:00,2020-12-31 00:00:00,2021-01-15 00:00:00
1,NaN,Subject to Bar under 8 CFR 208.13(c)(4) - No Exception Established,Case Receipts,1018,1893,1199,1384,1156,984,307,195,175,132,137,55,-,-,-,-,-,-,-,-,-,-,-,-,-
2,NaN,NaN,All Decisions,647,1479,1566,1197,880,1749,802,558,337,166,181,179,3,1,-,-,-,-,-,-,-,-,-,-,-
3,NaN,NaN,Fear Established (Y),198,469,629,425,246,594,371,238,136,81,92,87,3,1,-,-,-,-,-,-,-,-,-,-,-
4,NaN,NaN,Fear Not Established (N),332,794,728,607,454,919,384,292,179,69,71,84,-,-,-,-,-,-,-,-,-,-,-,-,-
5,NaN,NaN,Administratively Closed,117,216,209,165,180,236,47,28,22,16,18,8,-,-,-,-,-,-,-,-,-,-,-,-,-
6,NaN,Subject to Bar under 8 CFR 208.13(c)(4) - Exception Established,Case Receipts,20,29,28,30,34,40,14,11,8,5,15,1,-,-,-,-,-,-,-,-,-,-,-,-,-
7,NaN,NaN,All Decisions,12,23,21,16,18,43,38,30,22,3,19,6,-,-,-,-,-,-,-,-,-,-,-,-,-
8,NaN,NaN,Fear Established (Y),11,20,20,12,13,36,34,22,19,1,17,6,-,-,-,-,-,-,-,-,-,-,-,-,-
9,NaN,NaN,Fear Not Established (N),1,3,1,4,5,7,3,8,3,2,2,-,-,-,-,-,-,-,-,-,-,-,-,-,-


You can check additional tables by increasing the number in the `df[#]` call

**Review**:

So we now have a function to extract independent tables from a spreadsheet with multiple tables present. Below we have written additional functions to help us clean up these dataframes and get better column names, and target a specific table in the worksheet.

In [190]:
def get_time_cols_and_sort(df):
    """
    Helper function to sort dataframe columns that are dates in date increasing order

    Parameter:
        df: pandas dataframe


    Returns columns in sorted order
    """
    # Build a list of time ordered columns(columsn that are dates in seq. order)
    time_cols = []
    for i in df.columns:
        try:
            time_cols.append((i, pd.to_datetime(i)))
        except:
            continue

    properly_sorted_date_cols = sorted(
        time_cols,
        key=lambda x: x[1],
    )
    return properly_sorted_date_cols


def extract_sub_table(
    file_path,
    table_number=0,
    zero_symbols=["-"],
    primary_column=None,
    secondary_column=None,
    check_column_names=True,
):

    """
    Function to extract specific subtable from xlsx sheet, clean up columns and some data values
    and return that dataframe

    Parameter:
        file_path(str): path to excel file,
        table_number(int): number starting from 0 that indicates order table is in collection of tables,
        zero_symbols(list): values to replace with zero, default is '-'
        primary_column(str): first column that contains valid contextual information about the data (may be left null)
        secondary_column(str): second column that contains valid contextual information about the data (may be left null)
        check_column_names(bool): Ask user to submit primary and secondary column values through an input, if set to falses
                                    then at least primary_column must be populated.
    Returns columns in sorted order

    """

    # Grab the date info in the name to use as a label
    file_path_time_label = file_path.name.split("Congressional-Semi-Monthly-Report-")[
        1
    ].split("-to")[0]

    # Attempt to process this data
    #     try:

    # Use this function to segment the tables in the sheet
    test = parse_excel_sheet(file_path)

    # Grab the dataframes
    dfs = test[0]
    # Grab the table based on sequence number
    # TODO rename to generic name
    sub_table = dfs[table_number].copy()

    # Combine the columns data with the first row of data (which also contains column info) together
    cols = list(zip(sub_table.columns, sub_table.loc[0, :]))

    # Create clean column labels
    formatted_cols = []
    for col in cols:
        if isinstance(col[0], str):
            part1 = col[0]
            part2 = "" if pd.isnull(col[1]) else f"-{col[1]}"
            formatted_cols.append(f"{part1}{part2}")
        else:
            formatted_cols.append(f'{col[0].strftime("%m/%Y")}')

    # Assign the columns to the dataframe
    sub_table.columns = formatted_cols

    # Drop that first row since it is not data
    sub_table = sub_table.drop(0, axis=0)

    for symbol in zero_symbols:
        sub_table = sub_table.replace("-", 0)

    if check_column_names:
        print("Columns --> ", sub_table.columns)
        display(sub_table.head(8))
        primary_column = input("What is the primary column name ? ")
        secondary_column = input(
            "What is the secondary column name ? [Leave blank if none.] "
        )
        print(
            "Selected Primary: ",
            primary_column,
            "\n" "Selected Secondary: ",
            secondary_column,
        )

    # TODO assert something somewhere about the need for column names
    if secondary_column != "" and secondary_column is not None:
        sub_table[primary_column] = sub_table[primary_column].ffill()
        sub_table.insert(
            0,
            primary_column + "-" + secondary_column,
            sub_table[primary_column] + "-" + sub_table[secondary_column],
        )

        sub_table = sub_table.drop([primary_column, secondary_column], axis=1)
        primary_column = primary_column + "-" + secondary_column

    for col_idx in range(sub_table.shape[1]):
        try:
            sub_table.iloc[:, col_idx] = sub_table.iloc[:, col_idx].astype(int)
        except Exception:
            continue

    # Segment the numeric data and the contextual data
    numeric_data = sub_table.select_dtypes(include="int")
    non_numeric_data = sub_table.select_dtypes(exclude="int")
    # The data is presented bi-monthly - we want to sum
    # it up to get monthly totals. The dataframe has duplicate columns
    # names which represent the first and second half of the month
    # Here we sum up the duplicate columns to get monthly total
    numeric_data = numeric_data.groupby(level=0, axis=1).sum()
    # Recombine the numeric and contextual data
    sub_table = pd.concat([non_numeric_data, numeric_data], axis=1)

    # Add a flag so we know what timeframe this data covers
    sub_table["time_label"] = file_path_time_label
    properly_sorted_date_cols = get_time_cols_and_sort(sub_table)
    # Reorder the dataframe and get labels up front and then data time ordered
    sub_table = sub_table[
        [primary_column, "time_label"] + [i[0] for i in properly_sorted_date_cols]
    ]
    print("Likley Successfull Extracting Data - Check Results")
    return sub_table

## 3. Extracting and Cleaning Specific Tables

We will now walk through some examples of extracting a specific table from the spreadsheet, getting clean(er) column names and summarizing the data in a more efficient manner. 

**Some Python Context**<br>
In Python things are `"zero indexed"` meaning that the first element when counting is considered element 0. (0,1,2,3,4 vs. 1,2,3,4). So below we pass `table_number=0` to the table_number argument to grab the first table.

When we run this code it will ask us to input a primary and secondary column. Not all tables will have a secondary column but you must provide a primary column. The primary column is used to label the rows of data for the table being processed.

In a case where we have primary and secondary columns it generally means there is some sort of nested or merged field situation for the rows. The second table in the spreadsheet, **Credible Fear Cases Subject to Third Country Transit Bar - Interim Final Rule2** is an example of this.


**Example 1**

Let's grab the first table in the first sheet **All Credible Fear Cases**

For this example the primary column is "From-To". When you're asked to provide the primary column for this table, type `"From-To"` and press enter, for secondary column leave it blank and press enter.

Note: Casing matters, so in this example, for the primary column you must type From-To, anything else will result in an error

Now we will run it ... 

In [191]:
example1a = extract_sub_table(test_file_path, table_number=0)

Columns -->  Index(['Unnamed: 0', 'From-To', 'Unnamed: 2', '01/2020', '01/2020', '02/2020',
       '02/2020', '03/2020', '03/2020', '04/2020', '04/2020', '05/2020',
       '05/2020', '06/2020', '06/2020', '07/2020', '07/2020', '08/2020',
       '08/2020', '09/2020', '09/2020', '10/2020', '10/2020', '11/2020',
       '11/2020', '12/2020', '12/2020', '01/2021'],
      dtype='object')


,Unnamed: 0,From-To,Unnamed: 2,01/2020,01/2020,02/2020,02/2020,03/2020,03/2020,04/2020,04/2020,05/2020,05/2020,06/2020,06/2020,07/2020,07/2020,08/2020,08/2020,09/2020,09/2020,10/2020,10/2020,11/2020,11/2020,12/2020,12/2020,01/2021
1,NaN,Case Receipts,NaN,1877,2901,2097,2532,2055,1578,433,253,256,205,283,367,399,325,279,336,308,417,341,368,284,381,459,562,435
2,NaN,All Decisions,NaN,1316,2373,2419,2121,1866,2644,1002,704,467,214,251,313,240,470,412,276,239,259,499,446,242,335,410,388,571
3,NaN,Fear Established (Y),NaN,408,800,870,622,523,916,463,296,179,97,133,138,122,258,253,163,114,111,278,242,137,210,249,243,351
4,NaN,Fear Not Established (N),NaN,655,1236,1198,1199,992,1359,464,350,221,89,90,148,83,137,128,71,84,94,182,125,66,91,92,83,145
5,NaN,Administratively Closed,NaN,253,337,351,300,351,369,75,58,67,28,28,27,35,75,31,42,41,54,39,79,39,34,69,62,75


What is the primary column name ? From-To
What is the secondary column name ? [Leave blank if none.] 
Selected Primary:  From-To 
Selected Secondary:  
Likley Successfull Extracting Data - Check Results


In [192]:
example1a

,From-To,time_label,01/2020,02/2020,03/2020,04/2020,05/2020,06/2020,07/2020,08/2020,09/2020,10/2020,11/2020,12/2020,01/2021
1,Case Receipts,01-01-20,4778,4629,3633,686,461,650,724,615,725,709,665,1021,435
2,All Decisions,01-01-20,3689,4540,4510,1706,681,564,710,688,498,945,577,798,571
3,Fear Established (Y),01-01-20,1208,1492,1439,759,276,271,380,416,225,520,347,492,351
4,Fear Not Established (N),01-01-20,1891,2397,2351,814,310,238,220,199,178,307,157,175,145
5,Administratively Closed,01-01-20,590,651,720,133,95,55,110,73,95,118,73,131,75


Above we see the extracted data - comparing to the spreadsheet we can see we have successfully extracted the data. 

Now that we know the column information for this table we don't have to enter it manually. We can run the code and pass that information. See below.

In [167]:
example1b = extract_sub_table(
    test_file_path, primary_column="From-To", check_column_names=False, table_number=0
)

Likley Successfull Extracting Data - Check Results


In [168]:
example1b

,From-To,time_label,01/2020,02/2020,03/2020,04/2020,05/2020,06/2020,07/2020,08/2020,09/2020,10/2020,11/2020,12/2020,01/2021
1,Case Receipts,01-01-20,4778,4629,3633,686,461,650,724,615,725,709,665,1021,435
2,All Decisions,01-01-20,3689,4540,4510,1706,681,564,710,688,498,945,577,798,571
3,Fear Established (Y),01-01-20,1208,1492,1439,759,276,271,380,416,225,520,347,492,351
4,Fear Not Established (N),01-01-20,1891,2397,2351,814,310,238,220,199,178,307,157,175,145
5,Administratively Closed,01-01-20,590,651,720,133,95,55,110,73,95,118,73,131,75



Let's look at another example. 

**Example 2**

Using the same spreadsheet but a different table now.

For this example, 
* when you're asked to provide the primary column for this table, type "From-To" and press enter, 
* for secondary column type "Unnamed: 2" and press enter.

Note: Casing matters

In [169]:
example2a = extract_sub_table(test_file_path, table_number=1)

Columns -->  Index(['Unnamed: 0', 'From-To', 'Unnamed: 2', '01/2020', '01/2020', '02/2020',
       '02/2020', '03/2020', '03/2020', '04/2020', '04/2020', '05/2020',
       '05/2020', '06/2020', '06/2020', '07/2020', '07/2020', '08/2020',
       '08/2020', '09/2020', '09/2020', '10/2020', '10/2020', '11/2020',
       '11/2020', '12/2020', '12/2020', '01/2021'],
      dtype='object')


,Unnamed: 0,From-To,Unnamed: 2,01/2020,01/2020,02/2020,02/2020,03/2020,03/2020,04/2020,04/2020,05/2020,05/2020,06/2020,06/2020,07/2020,07/2020,08/2020,08/2020,09/2020,09/2020,10/2020,10/2020,11/2020,11/2020,12/2020,12/2020,01/2021
1,NaN,Subject to Bar under 8 CFR 208.13(c)(4) - No Exception Established,Case Receipts,1018,1893,1199,1384,1156,984,307,195,175,132,137,55,0,0,0,0,0,0,0,0,0,0,0,0,0
2,NaN,NaN,All Decisions,647,1479,1566,1197,880,1749,802,558,337,166,181,179,3,1,0,0,0,0,0,0,0,0,0,0,0
3,NaN,NaN,Fear Established (Y),198,469,629,425,246,594,371,238,136,81,92,87,3,1,0,0,0,0,0,0,0,0,0,0,0
4,NaN,NaN,Fear Not Established (N),332,794,728,607,454,919,384,292,179,69,71,84,0,0,0,0,0,0,0,0,0,0,0,0,0
5,NaN,NaN,Administratively Closed,117,216,209,165,180,236,47,28,22,16,18,8,0,0,0,0,0,0,0,0,0,0,0,0,0
6,NaN,Subject to Bar under 8 CFR 208.13(c)(4) - Exception Established,Case Receipts,20,29,28,30,34,40,14,11,8,5,15,1,0,0,0,0,0,0,0,0,0,0,0,0,0
7,NaN,NaN,All Decisions,12,23,21,16,18,43,38,30,22,3,19,6,0,0,0,0,0,0,0,0,0,0,0,0,0
8,NaN,NaN,Fear Established (Y),11,20,20,12,13,36,34,22,19,1,17,6,0,0,0,0,0,0,0,0,0,0,0,0,0


What is the primary column name ? From-To
What is the secondary column name ? [Leave blank if none.] Unnamed: 2
Selected Primary:  From-To 
Selected Secondary:  Unnamed: 2
Likley Successfull Extracting Data - Check Results


In [193]:
example2b = extract_sub_table(
    test_file_path,
    table_number=1,
    primary_column="From-To",
    secondary_column="Unnamed: 2",
    check_column_names=False,
)

Likley Successfull Extracting Data - Check Results


In [195]:
example2b

,From-To-Unnamed: 2,time_label,01/2020,02/2020,03/2020,04/2020,05/2020,06/2020,07/2020,08/2020,09/2020,10/2020,11/2020,12/2020,01/2021
1,Subject to Bar under 8 CFR 208.13(c)(4) - No Exception Established-Case Receipts,01-01-20,2911,2583,2140,502,307,192,0,0,0,0,0,0,0
2,Subject to Bar under 8 CFR 208.13(c)(4) - No Exception Established-All Decisions,01-01-20,2126,2763,2629,1360,503,360,4,0,0,0,0,0,0
3,Subject to Bar under 8 CFR 208.13(c)(4) - No Exception Established-Fear Established (Y),01-01-20,667,1054,840,609,217,179,4,0,0,0,0,0,0
4,Subject to Bar under 8 CFR 208.13(c)(4) - No Exception Established-Fear Not Established (N),01-01-20,1126,1335,1373,676,248,155,0,0,0,0,0,0,0
5,Subject to Bar under 8 CFR 208.13(c)(4) - No Exception Established-Administratively Closed,01-01-20,333,374,416,75,38,26,0,0,0,0,0,0,0
6,Subject to Bar under 8 CFR 208.13(c)(4) - Exception Established-Case Receipts,01-01-20,49,58,74,25,13,16,0,0,0,0,0,0,0
7,Subject to Bar under 8 CFR 208.13(c)(4) - Exception Established-All Decisions,01-01-20,35,37,61,68,25,25,0,0,0,0,0,0,0
8,Subject to Bar under 8 CFR 208.13(c)(4) - Exception Established-Fear Established (Y),01-01-20,31,32,49,56,20,23,0,0,0,0,0,0,0
9,Subject to Bar under 8 CFR 208.13(c)(4) - Exception Established-Fear Not Established (N),01-01-20,4,5,12,11,5,2,0,0,0,0,0,0,0
10,Subject to Bar under 8 CFR 208.13(c)(4) - Exception Established-Administratively Closed,01-01-20,0,0,0,1,0,0,0,0,0,0,0,0,0


Ok now we can move on to a more advanced topic, which is extracting the same sub table across several different excel files. 

## 4. Extracting and combining data from many sheets

We have many `.xlsx` spreadsheets, we want to parse the same table out of all of the spreadsheets and then create a summary dataset for that table across time. We have written code to allow us to do this. 

In [1]:
def attempt_to_collect_and_combine_all_subsheets(
    data_folder, table_number, primary_column, secondary_column, zero_symbols=["-"]
):

    """
    Function to combine all sub tables from many .xlsx spreadsheets into a single dataframe.

    Parameters:
        data_folder: folder with all copies of same type of uscis xlsx file,
        table_number: index of the table number in the workbook,
        primary_column(str): first column that contains valid contextual information about the data (may be left null)
        secondary_column(str): second column that contains valid contextual information about the data (may be left null)
        zero_symbols(list): values to replace with zero, default is ['-']

    Returns:
        Summary dataframe with all instances of that table aggregated together

    """

    all_available_subsheets = pd.DataFrame()
    for fp in Path(data_folder).iterdir():
        if fp.name.endswith("xlsx") and "$" not in fp.name:
            print("\n Processing: ", fp)
            try:
                tmp = extract_sub_table(
                    fp,
                    table_number=table_number,
                    primary_column=primary_column,
                    secondary_column=secondary_column,
                    check_column_names=False,
                    zero_symbols=zero_symbols,
                )
                all_available_subsheets = all_available_subsheets.append(tmp)
            except BaseException as e:
                print(f"ERROR - {e}")

    if secondary_column is not None and secondary_column != "":
        primary_column = primary_column + "-" + secondary_column

    # Reorder the columns
    all_available_subsheets = all_available_subsheets[
        [primary_column, "time_label"]
        + [i[0] for i in get_time_cols_and_sort(all_available_subsheets)]
    ]

    return all_available_subsheets.sort_values("time_label").reset_index(drop=True)

**Example 1**: All Credible Fear Cases

In [200]:
all_cfr = attempt_to_collect_and_combine_all_subsheets(
    rawfile_output_directory, 0, primary_column="From-To", secondary_column=None
)
all_cfr


 Processing:  ../data/raw_source_files/uscis-cfr-semi-monthly-reports/Congressional-Semi-Monthly-Report-04-01-20-to-04-15-21.xlsx
Likley Successfull Extracting Data - Check Results

 Processing:  ../data/raw_source_files/uscis-cfr-semi-monthly-reports/Congressional-Semi-Monthly-Report-09-01-20-to-09-15-21.xlsx
Likley Successfull Extracting Data - Check Results

 Processing:  ../data/raw_source_files/uscis-cfr-semi-monthly-reports/Congressional-Semi-Monthly-Report-06-01-20-to-06-15-21.xlsx
Likley Successfull Extracting Data - Check Results

 Processing:  ../data/raw_source_files/uscis-cfr-semi-monthly-reports/Congressional-Semi-Monthly-Report-01-01-20-to-01-15-21.xlsx
Likley Successfull Extracting Data - Check Results

 Processing:  ../data/raw_source_files/uscis-cfr-semi-monthly-reports/Congressional-Semi-Monthly-Report-10-01-20-to-10-15-21.xlsx
Likley Successfull Extracting Data - Check Results

 Processing:  ../data/raw_source_files/uscis-cfr-semi-monthly-reports/Congressional-Semi-

,From-To,time_label,01/2020,02/2020,03/2020,04/2020,05/2020,06/2020,07/2020,08/2020,09/2020,10/2020,11/2020,12/2020,01/2021,02/2021,03/2021,04/2021,05/2021,06/2021,07/2021,08/2021,09/2021,10/2021,11/2021
0,Administratively Closed,01-01-20,590.0,651.0,720.0,133.0,95.0,55.0,110.0,73.0,95.0,118.0,73,131,75,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Fear Not Established (N),01-01-20,1891.0,2397.0,2351.0,814.0,310.0,238.0,220.0,199.0,178.0,307.0,157,175,145,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Fear Established (Y),01-01-20,1208.0,1492.0,1439.0,759.0,276.0,271.0,380.0,416.0,225.0,520.0,347,492,351,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,All Decisions,01-01-20,3689.0,4540.0,4510.0,1706.0,681.0,564.0,710.0,688.0,498.0,945.0,577,798,571,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Case Receipts,01-01-20,4778.0,4629.0,3633.0,686.0,461.0,650.0,724.0,615.0,725.0,709.0,665,1021,435,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Administratively Closed,02-01-20,NaN,651.0,720.0,133.0,95.0,55.0,110.0,73.0,95.0,118.0,73,131,126,104.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Fear Not Established (N),02-01-20,NaN,2397.0,2348.0,814.0,310.0,238.0,219.0,197.0,178.0,307.0,153,172,287,244.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Fear Established (Y),02-01-20,NaN,1492.0,1442.0,759.0,276.0,271.0,381.0,417.0,225.0,520.0,352,495,634,540.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,All Decisions,02-01-20,NaN,4540.0,4510.0,1706.0,681.0,564.0,710.0,687.0,498.0,945.0,578,798,1047,888.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Case Receipts,02-01-20,NaN,4629.0,3632.0,686.0,461.0,650.0,724.0,615.0,725.0,709.0,665,1021,1279,1174.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Above we see information on each file that was attempted to be processed and then the data extracted from the sheets. We see that there was an error processing the August file, something is strange with the formatting and is causing this error. 

As for the data, this code just extracts each table from each .xlsx file and then appends them together to create one large dataframe. In this file we have many values across time for different pieces of information. For instance for `Case Receipts` we have values for every .xlsx file (01-01-20, 02-01-20, etc). This means that for a specific month that USCIS is reporting information (such as 01/2020) we may have different results based on when USCIS reported. 

We can fix this in a few cells and create a final summary spreadsheet. 

**Example 2**: Credible Fear Cases Subject to Third Country Transit Bar - Interim Final Rule

In [201]:
cfr_3rd_country_bar = attempt_to_collect_and_combine_all_subsheets(
    rawfile_output_directory, 1, primary_column="From-To", secondary_column="Unnamed: 2"
)


 Processing:  ../data/raw_source_files/uscis-cfr-semi-monthly-reports/Congressional-Semi-Monthly-Report-04-01-20-to-04-15-21.xlsx
Likley Successfull Extracting Data - Check Results

 Processing:  ../data/raw_source_files/uscis-cfr-semi-monthly-reports/Congressional-Semi-Monthly-Report-09-01-20-to-09-15-21.xlsx
Likley Successfull Extracting Data - Check Results

 Processing:  ../data/raw_source_files/uscis-cfr-semi-monthly-reports/Congressional-Semi-Monthly-Report-06-01-20-to-06-15-21.xlsx
Likley Successfull Extracting Data - Check Results

 Processing:  ../data/raw_source_files/uscis-cfr-semi-monthly-reports/Congressional-Semi-Monthly-Report-01-01-20-to-01-15-21.xlsx
Likley Successfull Extracting Data - Check Results

 Processing:  ../data/raw_source_files/uscis-cfr-semi-monthly-reports/Congressional-Semi-Monthly-Report-10-01-20-to-10-15-21.xlsx
Likley Successfull Extracting Data - Check Results

 Processing:  ../data/raw_source_files/uscis-cfr-semi-monthly-reports/Congressional-Semi-

In [203]:
cfr_3rd_country_bar

,From-To-Unnamed: 2,time_label,01/2020,02/2020,03/2020,04/2020,05/2020,06/2020,07/2020,08/2020,09/2020,10/2020,11/2020,12/2020,01/2021,02/2021,03/2021,04/2021,05/2021,06/2021,07/2021,08/2021,09/2021,10/2021,11/2021
0,Subject to Bar under 8 CFR 208.13(c)(4) - Exception Established-Administratively Closed,01-01-20,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Subject to Bar under 8 CFR 208.13(c)(4) - No Exception Established-Fear Established (Y),01-01-20,667.0,1054.0,840.0,609.0,217.0,179.0,4.0,0.0,0.0,0.0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Subject to Bar under 8 CFR 208.13(c)(4) - No Exception Established-Fear Not Established (N),01-01-20,1126.0,1335.0,1373.0,676.0,248.0,155.0,0.0,0.0,0.0,0.0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Subject to Bar under 8 CFR 208.13(c)(4) - No Exception Established-Administratively Closed,01-01-20,333.0,374.0,416.0,75.0,38.0,26.0,0.0,0.0,0.0,0.0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Subject to Bar under 8 CFR 208.13(c)(4) - Exception Established-Case Receipts,01-01-20,49.0,58.0,74.0,25.0,13.0,16.0,0.0,0.0,0.0,0.0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,Subject to Bar under 8 CFR 208.13(c)(4) - No Exception Established-Fear Established (Y),11-01-20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
76,Subject to Bar under 8 CFR 208.13(c)(4) - No Exception Established-All Decisions,11-01-20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0.0,2.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
77,Subject to Bar under 8 CFR 208.13(c)(4) - Exception Established-Fear Not Established (N),11-01-20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
78,Subject to Bar under 8 CFR 208.13(c)(4) - No Exception Established-Fear Not Established (N),11-01-20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## 5.  Summarizing the Data

Now that we have all the data we need, let's create a summary table that just has the most recent data for each category

In [205]:
def create_final_summary(df, column_label):

    """
    Function that converts dataframe of USCIS data sub table from many different months of spreadsheets
    and then keeps the most recently reported data by USCIS while discarding older or missing data.

    Parameters:
        df(pd.DataFrame): dataframe of USCIS data sub tables,
        column_label(str): column label to use for summarizing data

    """

    compiled_rows = []

    info_elems = df[column_label].unique()

    properly_sorted_date_cols = get_time_cols_and_sort(df)

    for elem in info_elems:
        tmp = df[df[column_label] == elem]
        compiled_rows.append(
            tmp[[i[0] for i in properly_sorted_date_cols]].ffill().iloc[-1]
        )

    out = pd.concat(compiled_rows, axis=1)

    out.columns = info_elems

    out = out.T.reset_index(drop=False)
    out = out.rename(columns={"index": column_label})
    out = out.sort_values(column_label)
    return out

**Example 1**

In [208]:
create_final_summary(all_cfr, "From-To")

,From-To,01/2020,02/2020,03/2020,04/2020,05/2020,06/2020,07/2020,08/2020,09/2020,10/2020,11/2020,12/2020,01/2021,02/2021,03/2021,04/2021,05/2021,06/2021,07/2021,08/2021,09/2021,10/2021,11/2021
0,Administratively Closed,590.0,651.0,720.0,133.0,95.0,55.0,110.0,73.0,95.0,118.0,73.0,131.0,127.0,156.0,282.0,179.0,209.0,264.0,512.0,171.0,157.0,114.0,110.0
3,All Decisions,3689.0,4540.0,4510.0,1706.0,681.0,564.0,710.0,689.0,498.0,945.0,578.0,799.0,1047.0,1631.0,2884.0,3293.0,4466.0,6206.0,7616.0,7075.0,7431.0,6358.0,3323.0
4,Case Receipts,4778.0,4629.0,3632.0,686.0,461.0,651.0,724.0,612.0,725.0,709.0,664.0,1020.0,1278.0,2503.0,5234.0,6611.0,7440.0,10103.0,8535.0,8707.0,6349.0,8186.0,3858.0
2,Fear Established (Y),1208.0,1492.0,1442.0,760.0,276.0,275.0,382.0,418.0,225.0,522.0,352.0,496.0,639.0,1043.0,1856.0,2199.0,3047.0,4400.0,5278.0,5286.0,4889.0,4154.0,2304.0
1,Fear Not Established (N),1891.0,2397.0,2348.0,813.0,310.0,234.0,218.0,198.0,178.0,305.0,153.0,172.0,281.0,432.0,746.0,915.0,1218.0,1622.0,1942.0,1721.0,2489.0,2183.0,965.0


**Example 2**

In [211]:
create_final_summary(cfr_3rd_country_bar, "From-To-Unnamed: 2")

,From-To-Unnamed: 2,01/2020,02/2020,03/2020,04/2020,05/2020,06/2020,07/2020,08/2020,09/2020,10/2020,11/2020,12/2020,01/2021,02/2021,03/2021,04/2021,05/2021,06/2021,07/2021,08/2021,09/2021,10/2021,11/2021
0,Subject to Bar under 8 CFR 208.13(c)(4) - Exception Established-Administratively Closed,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,Subject to Bar under 8 CFR 208.13(c)(4) - Exception Established-All Decisions,35.0,37.0,61.0,68.0,25.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,Subject to Bar under 8 CFR 208.13(c)(4) - Exception Established-Case Receipts,49.0,58.0,73.0,25.0,13.0,16.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,Subject to Bar under 8 CFR 208.13(c)(4) - Exception Established-Fear Established (Y),31.0,32.0,49.0,56.0,20.0,23.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
7,Subject to Bar under 8 CFR 208.13(c)(4) - Exception Established-Fear Not Established (N),4.0,5.0,12.0,11.0,5.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Subject to Bar under 8 CFR 208.13(c)(4) - No Exception Established-Administratively Closed,333.0,374.0,416.0,75.0,38.0,26.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,Subject to Bar under 8 CFR 208.13(c)(4) - No Exception Established-All Decisions,2126.0,2763.0,2629.0,1360.0,503.0,360.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
9,Subject to Bar under 8 CFR 208.13(c)(4) - No Exception Established-Case Receipts,2911.0,2583.0,2140.0,502.0,307.0,192.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Subject to Bar under 8 CFR 208.13(c)(4) - No Exception Established-Fear Established (Y),667.0,1054.0,843.0,610.0,217.0,182.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,Subject to Bar under 8 CFR 208.13(c)(4) - No Exception Established-Fear Not Established (N),1126.0,1335.0,1370.0,675.0,248.0,152.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# End